In [247]:
import pandas as pd
from IPython.display import display, HTML

import numpy as np

In [248]:
df = pd.read_csv('sample_data/california_housing_train.csv').iloc[:30]
df = df.reset_index()
df['Categoria'] = np.append(np.repeat('A', 15), np.repeat('B', 15))

In [249]:
#Define as regras para colorir a tabela incluindo os indices e os cabeçalhos

#th: edita os cabeçalhos e os índices
#tr: edita os valores da tabela
#'': edita a tabela como um todo
styles = [{'selector': 'th', 'props': [('background-color', 'black'), ('color', 'white'),
                                       ('border-style','solid'), ('border-width','0px'),
                                       ('font-size', '12pt')]},
          {'selector': 'tr', 'props': [('background-color', 'white'), ('color', 'black'),
                                       ('font-size', '12pt')]},
          {'selector': '', 'props': [(('width', '90%'))]}]

df_styler = df.style.set_table_styles(styles)

In [250]:
#Define as regras para colorir as valores da tabela (não colore os indices e os cabeçalhos)

def f(x):
    if x.name == 'median_house_value':
        def g(i, v):
            if i >= 25 and v > 60000:
                return ('background-color: red' + '; color: white')
            else:
                ('background-color: white' + '; color: black')
        return [g(i, v) for i, v in x.iteritems()]
    else:
        return [('background-color: white' + '; color: black') for i, v in x.iteritems()]

#faz um apply coluna a a coluna, então x é um pd.Series (e possui os indices) e x.name pega o nome da coluna do loop
df_cores = df.apply(lambda x: f(x), axis = 0)

def apply_color(x):
    return df_cores

df_styler = df_styler.apply(apply_color, axis = None)

In [251]:
#Oculta os índices
#df_styler = df_styler.hide_index()

#Centraliza os textos
df_styler = df_styler.set_properties(**{'text-align': 'center'})

In [252]:
html = df_styler.to_html()

#Troca o id da tabela
def troca_id_tabela(html, id_tabela):
    str_find = '<table id="'
    id_ini = html.find('<table id="')
    id_fim = html[id_ini:].find('">')
    id_atual = html[id_ini+len(str_find):id_ini+id_fim]
    html = html.replace(id_atual, id_tabela)
    return html

html = troca_id_tabela(html, id_tabela = 'tabela_dados')

In [284]:
def adiciona_buscas(html, id_tabela):
    inputs_code = '''
Categoria: <select id='input_selecao'>
<option value = "none">Geral</option>
<option value = "A">A</option>
<option value = "B">B</option>
</select><br><br>
Index: <input type="text" id="input_texto" placeholder="Busca por index"><br><br>
<input type="button" value="Buscar" onclick="funcaoBusca()"><br><br>
    '''

    script_code = '''
<script>
function funcaoBusca() {
    var input_texto, filter, input_selecao, selecao;
    input_texto = document.getElementById("input_texto");
    filter = input_texto.value;
    input_selecao = document.getElementById("input_selecao");
    selecao = input_selecao.value;

    var table, thead_ths, tr;
    table = document.getElementById("id_tabela");
    thead_ths = table.tHead.getElementsByTagName("tr")[0].getElementsByTagName("th");
    var j, col_name, col_id_texto, col_id_selecao;
    col_id_texto = 0;
    col_id_selecao = 0;
    for (j = 0; j < thead_ths.length; j++){
        col_name = thead_ths[j].innerHTML;
        if(col_name == "index"){
            col_id_texto = j;
        }
        if(col_name == "Categoria"){
            col_id_selecao = j;
        }
    }

    tr = table.getElementsByTagName("tr");
    var col_id, vlFilter, i, td, txtValue, display_status;

    // Loop em todas as linhas para deixar todas com display ativo
    for (i = 0; i < tr.length; i++) {
        tr[i].style.display = "";
    }

    // Loop tem todas as linhas escondendo as que não batem com o filtro
    if(filter != ''){
        col_id = col_id_texto;
        vlFilter = filter;
        for (i = 0; i < tr.length; i++) {
            shift = tr[i].getElementsByTagName("th").length;
            td = tr[i].getElementsByTagName("td")[col_id - shift];
            display_status = tr[i].style.display;
            if (td) {
                txtValue = td.textContent || td.innerText;
                if (txtValue == vlFilter) {
                    tr[i].style.display = display_status;
                } else {
                    tr[i].style.display = "none";
                }
            }
        }
    }

    // Loop tem todas as linhas escondendo as que não batem com o filtro
    if(selecao != 'none'){
        col_id = col_id_selecao;
        vlFilter = selecao;
        for (i = 0; i < tr.length; i++) {
            shift = tr[i].getElementsByTagName("th").length;
            td = tr[i].getElementsByTagName("td")[col_id - shift];
            display_status = tr[i].style.display;
            if (td) {
                txtValue = td.textContent || td.innerText;
                if (txtValue == vlFilter) {
                    tr[i].style.display = display_status;
                } else {
                    tr[i].style.display = "none";
                }
            }
        }
    }

}
</script>
    '''
    script_code = script_code.replace('id_tabela', id_tabela)
    return inputs_code + '\n' + html + '\n' + script_code

html = adiciona_buscas(html, id_tabela = 'tabela_dados')

In [285]:
#HTML(adiciona_buscas(html, id_tabela = 'tabela_dados'))

In [287]:
with open('dados_vis.html', 'w') as file:
    file.write(html)